# Lesson 03
## Парсинг HTML. BeautifulSoup, MongoDB

## 1)Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
def parsing_hh(vacancy):

    vacancy_date = []
    
    params = {
        'text': vacancy, \
        'search_field': 'name', \
        'items_on_page': '100', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://hh.ru/search/vacancy'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        
        page_block = parsed_html.find('div', {'data-qa': 'pager-block'})
        if not page_block:
            last_page = '1'
        else:
            last_page = int(page_block.find_all('a', {'class': 'HH-Pager-Control'})[-2].getText())
    
    for page in range(0, last_page):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            
            vacancy_items = parsed_html.find('div', {'data-qa': 'vacancy-serp__results'}) \
                                        .find_all('div', {'class': 'vacancy-serp-item'})
                
            for item in vacancy_items:
                vacancy_date.append(item_hh(item))
                
    return vacancy_date

In [3]:
def item_hh(item):

    vacancy_date = {}
    
 
    vacancy_name = item.find('a', {'class': 'bloko-link HH-LinkModifier'}) \
                        .getText() \
                        .replace(u'\xa0', u' ')
    
    vacancy_date['vacancy_name'] = vacancy_name
    
    
    salary = item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary = salary.getText() \
                        .replace(u'\xa0', u'')
        
        salary = re.split(r'\s|-', salary)
        
        if salary[0] == 'до':
            salary_min = None
            salary_max = int(salary[1])
        elif salary[0] == 'от':
            salary_min = int(salary[1])
            salary_max = None
        else:
            salary_min = int(salary[0])
            salary_max = int(salary[1])            
        
        salary_currency = salary[2]
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    
    vacancy_link = item.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
    
    vacancy_date['vacancy_link'] = vacancy_link 
    

    vacancy_date['site'] = 'hh.ru'
    
    return vacancy_date

In [4]:
def parsing_superjob(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy, \
        'profession_only': '1', \
        'geo[c][0]': '15', \
        'geo[c][1]': '1', \
        'geo[c][2]': '9', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})
                        
            for item in vacancy_items:
                vacancy_date.append(item_superjob(item))
                
    return vacancy_date

In [5]:
def item_superjob(item):

    vacancy_date = {}
    
 
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name
    

   

    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                  .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        is_check_sarary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                .getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        if is_check_sarary == 'до' or len(salary) == 2:
            salary_min = None
            salary_max = int(salary[0].getText() \
                                        .replace(u'\xa0', u''))
        elif is_check_sarary == 'от':
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = None
        else:
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = int(salary[2].getText() \
                                         .replace(u'\xa0', u''))     
            
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
    

    vacancy_link = item.find_all('a')
    
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']
    
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link }'
    

    vacancy_date['site'] = 'www.superjob.ru'
    
    return vacancy_date

In [6]:
def parsing_vacancy(vacancy):
        
    vacancy_date = []
    vacancy_date.extend(parsing_hh(vacancy))
    vacancy_date.extend(parsing_superjob(vacancy))
    
    df = pd.DataFrame(vacancy_date)

    return df

In [7]:
vacancy = 'Python'
df = parsing_vacancy(vacancy)

In [8]:
df.loc[(df['site']== 'hh.ru')&(df['salary_min'].notnull()),:].head(3)

,vacancy_name,salary_min,salary_max,salary_currency,vacancy_link,site
0,Python Developer,2500.0,NaN,USD,https://hh.ru/vacancy/35095020?query=Python,hh.ru
3,Программист Python,95000.0,NaN,руб.,https://hh.ru/vacancy/36293188?query=Python,hh.ru
4,Python developer,150000.0,200000.0,руб.,https://hh.ru/vacancy/36267290?query=Python,hh.ru


In [146]:
df.to_csv('homework_3_malik')

In [147]:
import json
from pymongo import MongoClient
from pprint import pprint

vacancy_csv = pd.read_csv('homework_3.csv')
vacancy.dropna(inplace=True)


try: 
    client = MongoClient('localhost', 27017)
    print("Connected successfully!!!") 
except:   
    print("Could not connect to MongoDB") 
    
db = client['vacancy_HH_SUPERJOB']

collection = db['collection']


Connected successfully!!!


In [148]:
json= [{'vacancy_name': a,'salary_min': int(b),'salary_max': int(c), 'vacancy_link': d} for a, b,c,d in zip(vacancy['vacancy_name'], vacancy['salary_min'],vacancy['salary_max'], vacancy['vacancy_link'])]
pprint(json)

[{'salary_max': 2000,
  'salary_min': 1700,
  'vacancy_link': 'https://hh.ru/vacancy/36222549?query=Python',
  'vacancy_name': 'Fullstack Python Developer'},
 {'salary_max': 5000,
  'salary_min': 2500,
  'vacancy_link': 'https://hh.ru/vacancy/36232273?query=Python',
  'vacancy_name': 'Software developer Python (Германия)'},
 {'salary_max': 110000,
  'salary_min': 70000,
  'vacancy_link': 'https://hh.ru/vacancy/35950905?query=Python',
  'vacancy_name': 'Разработчик Python'},
 {'salary_max': 170000,
  'salary_min': 150000,
  'vacancy_link': 'https://hh.ru/vacancy/35590347?query=Python',
  'vacancy_name': 'Разработчик Python'},
 {'salary_max': 40000,
  'salary_min': 30000,
  'vacancy_link': 'https://hh.ru/vacancy/36160045?query=Python',
  'vacancy_name': 'Python/Django разработчик (part time)'},
 {'salary_max': 800000,
  'salary_min': 150000,
  'vacancy_link': 'https://hh.ru/vacancy/35913318?query=Python',
  'vacancy_name': 'Backend Python Developer'},
 {'salary_max': 120000,
  'salary_mi

  'vacancy_link': 'https://hh.ru/vacancy/36088274?query=Python',
  'vacancy_name': 'Full-stack .NET Core / Python Software Engineer '
                  '(Junior/Middle)'},
 {'salary_max': 100000,
  'salary_min': 20000,
  'vacancy_link': 'https://hh.ru/vacancy/36045597?query=Python',
  'vacancy_name': 'Python/Django and Big Data Expert'},
 {'salary_max': 3000,
  'salary_min': 600,
  'vacancy_link': 'https://hh.ru/vacancy/36003291?query=Python',
  'vacancy_name': 'Python-программист (Разработчик)'},
 {'salary_max': 18000,
  'salary_min': 14500,
  'vacancy_link': 'https://hh.ru/vacancy/35977913?query=Python',
  'vacancy_name': 'Разработчик Python / Верстальщик WEB'},
 {'salary_max': 130000,
  'salary_min': 80000,
  'vacancy_link': 'https://hh.ru/vacancy/30548119?query=Python',
  'vacancy_name': 'Middle / Senior Python developer'},
 {'salary_max': 120000,
  'salary_min': 60000,
  'vacancy_link': 'https://hh.ru/vacancy/34266667?query=Python',
  'vacancy_name': 'Fullstack-разработчик (PHP / 

In [149]:
collection.insert_many(json)

## КАК из JSON сделать DATAFRAME ?   Как из DATAFRAME сделать Json без ZIP?

In [150]:
print(type(json))  # "ЭТО JSON" ?

<class 'list'>


## 2)Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы

In [151]:
vacancylist= []
def salary(min,max):
    for i in collection.find(( { '$and': [ { 'salary_min': { '$gt': min } }, { 'salary_max': { '$gt': max } } ] } ) ):
            #pprint(i)
            vacancylist.append(i)
                        
salary(min= 1000, max= 2000)

vacancy_dataframe =  pd.DataFrame(vacancylist)
#pprint(vacacancylist)

vacancy_dataframe

,_id,vacancy_name,salary_min,salary_max,vacancy_link
0,5e723bd6924a242929711865,Software developer Python (Германия),2500,5000,https://hh.ru/vacancy/36232273?query=Python
1,5e723bd6924a242929711866,Разработчик Python,70000,110000,https://hh.ru/vacancy/35950905?query=Python
2,5e723bd6924a242929711867,Разработчик Python,150000,170000,https://hh.ru/vacancy/35590347?query=Python
3,5e723bd6924a242929711868,Python/Django разработчик (part time),30000,40000,https://hh.ru/vacancy/36160045?query=Python
4,5e723bd6924a242929711869,Backend Python Developer,150000,800000,https://hh.ru/vacancy/35913318?query=Python
...,...,...,...,...,...
333,5e723cdf924a2429297119bc,Ведущий программист Python,160000,245000,https://hh.ru/vacancy/33757717?query=Python
334,5e723cdf924a2429297119bd,Ростелеком,150000,300000,https://www.superjob.ru/clients/rostelekom-241...
335,5e723cdf924a2429297119be,Разработчик Python,170000,200000,https://www.superjob.ru/vakansii/razrabotchik-...
336,5e723cdf924a2429297119bf,Ведущий разработчик Python,150000,300000,https://www.superjob.ru/vakansii/veduschij-raz...


print(type(json))   ЭТО JSON?  почему VacancyList начинается и заканчивается без [  ] в отлчие от JSON ??

## Как из этого сделать JSON ?

In [152]:
data = pd.read_csv("homework_3.csv")    
data.dropna(inplace = True) 
data_dict = data.to_dict() 
#pprint(data_dict)
print(type(data_dict))

<class 'dict'>


In [153]:
print(type(json))  # "ЭТО JSON" ?
print(type(vacancylist)) # 'ЭТО JSON?'

<class 'list'>
<class 'list'>
